In [ ]:
import pandas as pd
from fuzzywuzzy import process, fuzz
import folium

# Charger les données
df_top_50 = pd.read_csv("top_50_festivals.csv")
df_france = pd.read_csv("festivals_en_France.csv", delimiter=";", encoding="utf-8-sig") # la première colonne de ce fichier csv est mal formatée 

# Renommer la colonne avec le BOM dans festivals_en_France
df_france.rename(columns={"﻿Nom du festival": "Nom du festival"}, inplace=True)

# Normaliser les noms et les communes
df_top_50["Nom"] = df_top_50["Nom"].str.upper().str.strip()
df_top_50["Lieu"] = df_top_50["Lieu"].str.upper().str.strip()

df_france["Nom du festival"] = df_france["Nom du festival"].str.upper().str.strip()
df_france["Commune principale de déroulement"] = (
    df_france["Commune principale de déroulement"].str.upper().str.strip()
)

# Afficher un aperçu
print("Aperçu des festivals Top 50 :")
print(df_top_50.head())

print("\nAperçu des festivals en France :")
print(df_france.head())


Aperçu des festivals Top 50 :
   Classement                              Nom                           Lieu  \
0           1                FESTIVAL DE NIMES                    NIMES\n(30)   
1           2            PRINTEMPS DE PEROUGES  SAINT MAURICE DE REMENS\n(01)   
2           3  FESTIVAL LES NUITS DE FOURVIERE                     LYON\n(69)   
3           4          FESTIVAL DE CARCASSONNE              CARCASSONNE\n(11)   
4           5                 LUXEXPO OPEN AIR       LUXEMBOURG\n(LUXEMBOURG)   

                                                Lien  
0  https://infoconcert.com/festival/festival-de-n...  
1  https://infoconcert.com/festival/printemps-de-...  
2  https://infoconcert.com/festival/festival-les-...  
3  https://infoconcert.com/festival/festival-de-c...  
4  https://infoconcert.com/festival/luxexpo-open-...  

Aperçu des festivals en France :
            Nom du festival Envergure territoriale  \
0          FESTIVAL ANDALOU              Régionale   
1  FESTIVAL

In [2]:
# Trouver le meilleur match basé sur le nom ou la commune
def find_best_match(row, df_reference, key_column, secondary_column, score_threshold=80):
    # Correspondance par nom
    best_match = process.extractOne(row["Nom"], df_reference[key_column], scorer=fuzz.token_sort_ratio)
    if best_match and best_match[1] >= score_threshold:
        return best_match[0]
    
    # Si pas de correspondance par nom, chercher par commune
    if secondary_column in row and pd.notna(row[secondary_column]):
        best_match_commune = process.extractOne(
            row[secondary_column], df_reference[secondary_column], scorer=fuzz.token_sort_ratio
        )
        if best_match_commune and best_match_commune[1] >= score_threshold:
            return best_match_commune[0]
    
    # Aucun match trouvé
    return None

# Ajouter une colonne avec le meilleur match
df_top_50["Festival match"] = df_top_50.apply(
    lambda row: find_best_match(
        row, 
        df_france, 
        "Nom du festival", 
        "Commune principale de déroulement"
    ), 
    axis=1
)

# Afficher un aperçu des correspondances
print("\nCorrespondances trouvées :")
print(df_top_50[["Nom", "Lieu", "Festival match"]].head())



Correspondances trouvées :
                               Nom                           Lieu  \
0                FESTIVAL DE NIMES                    NIMES\n(30)   
1            PRINTEMPS DE PEROUGES  SAINT MAURICE DE REMENS\n(01)   
2  FESTIVAL LES NUITS DE FOURVIERE                     LYON\n(69)   
3          FESTIVAL DE CARCASSONNE              CARCASSONNE\n(11)   
4                 LUXEXPO OPEN AIR       LUXEMBOURG\n(LUXEMBOURG)   

            Festival match  
0      FESTIVAL DE NÉOULES  
1     PRINTEMPS DE PAROLES  
2   LES NUITS DE FOURVIÈRE  
3  FESTIVAL DE CARCASSONNE  
4                     None  


In [3]:
# Fusionner les données basées sur les correspondances fuzzy
df_top_50["Festival match"] = df_top_50["Festival match"].fillna("")

df_merged = pd.merge(
    df_france,
    df_top_50,
    left_on="Nom du festival",
    right_on="Festival match",
    how="left"
)

# Vérifier l'aperçu des données fusionnées
print("\nAperçu des données fusionnées :")
print(df_merged.head())



Aperçu des données fusionnées :
            Nom du festival Envergure territoriale  \
0          FESTIVAL ANDALOU              Régionale   
1  FESTIVAL RÊVES D'ENFANTS                    NaN   
2      PRINTEMPS DE PAROLES                    NaN   
3                 PHARAONIC                    NaN   
4           SARCUS FESTIVAL                    NaN   

  Région principale de déroulement Département principal de déroulement  \
0       Provence-Alpes-Côte d'Azur                             Vaucluse   
1       Provence-Alpes-Côte d'Azur                         Hautes-Alpes   
2                    Île-de-France                       Seine-et-Marne   
3             Auvergne-Rhône-Alpes                               Savoie   
4              Centre-Val de Loire                       Indre-et-Loire   

  Commune principale de déroulement  \
0                           AVIGNON   
1                          BRIANÇON   
2                BUSSY-SAINT-MARTIN   
3                          CHAMBÉRY

In [5]:
# Identifier les festivals sans coordonnées géographiques
missing_coords = df_merged[df_merged[["Latitude", "Longitude"]].isna()]
print("\nFestivals sans coordonnées géographiques :")
print(missing_coords[["Nom du festival", "Commune principale de déroulement"]])



Festivals sans coordonnées géographiques :
     Nom du festival Commune principale de déroulement
0                NaN                               NaN
1                NaN                               NaN
2                NaN                               NaN
3                NaN                               NaN
4                NaN                               NaN
...              ...                               ...
7278             NaN                               NaN
7279             NaN                               NaN
7280             NaN                               NaN
7281             NaN                               NaN
7282             NaN                               NaN

[7283 rows x 2 columns]


In [4]:
# Extraire latitude et longitude
df_merged[["Latitude", "Longitude"]] = (
    df_merged["Géocodage xy"]
    .str.split(",", expand=True)
    .astype(float)
)

# Créer une carte centrée sur la France
map_festivals = folium.Map(location=[46.603354, 1.888334], zoom_start=6)

# Ajouter tous les festivals (points standards en bleu)
for _, row in df_merged.iterrows():
    if pd.notna(row["Classement"]):  # Festivals populaires
        folium.Marker(
            location=[row["Latitude"], row["Longitude"]],
            popup=f"#{int(row['Classement'])} {row['Nom']} - {row['Lieu']}",
            icon=folium.Icon(color="red", icon="star"),
        ).add_to(map_festivals)
    else:  # Autres festivals
        folium.CircleMarker(
            location=[row["Latitude"], row["Longitude"]],
            radius=5,
            color="blue",
            fill=True,
            fill_color="blue",
            fill_opacity=0.6,
            popup=row["Nom du festival"],
        ).add_to(map_festivals)

# Sauvegarder la carte
map_festivals.save("carte_festivals_france.html")
print("Carte sauvegardée sous 'carte_festivals_france.html'.")


ValueError: Location values cannot contain NaNs.